In [ ]:
import os, sys

workspace = '/content/drive/MyDrive/Colab Notebooks/Text2Struct/'
print(os.path.exists(workspace))

modules_dir = workspace + 'modules/'
sys.path.insert(0, modules_dir)

data_dir = workspace + 'dataset/annotated/thrombectomy_0-99/'
out_dir = workspace + 'dataset/combined/'

True


In [ ]:
import brat_parser
import pathlib
import nltk
nltk.download('punkt')
import numpy as np
import pickle

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Annotation processing

In [ ]:
def spans(txt):
    # tokens = nltk.word_tokenize(txt)  # note: it wrongly changes quotes
    tokens = txt.split()
    offset = 0
    for token in tokens:
        offset = txt.find(token, offset)
        yield token, offset, offset+len(token)
        assert token == txt[offset:offset+len(token)]
        offset += len(token)

# ## Checking
# s = 'Following time intervals were shortened in comparison to 2016 : " hospital arrival - GP " ( 77 vs. 53 min ; p < 0.0001 ).'
# print(s)
# for token in spans(s):
#     print(token)
#     # assert token[0]==s[token[1]:token[2]]

In [ ]:
# folder_list = [os.path.join(data_dir, f) for f in sorted(os.listdir(data_dir)) 
#                if not os.path.isfile(os.path.join(data_dir, f))]
# folder_list;

# folder_dir = folder_list[51]
# sentence_names = [pathlib.Path(f).stem for f in sorted(os.listdir(folder_dir))
#                  if pathlib.Path(f).suffix == '.txt']
# sentence_names

# sentence_name = sentence_names[5]

# sentence_name

In [ ]:
def n_upper_chars(string):
    return sum(1 for c in string if c.isupper())

In [ ]:
# ## Extract content
# txt_path = os.path.join(folder_dir, sentence_name + '.txt')
# f = open(txt_path, 'r')
# content = f.read()
# print(content)

# tokens_and_spans = [t for t in spans(content)]
# print(tokens_and_spans)

# # tokens = [t[0] for t in tokens_and_spans]
# tokens = []
# for t in tokens_and_spans:
#   token = t[0]
#   if token[0].isupper() and n_upper_chars(token) < 2:
#     token = token.lower()
#   tokens.append(token)

# tokens = np.array(tokens)
# print(tokens)

In [ ]:
# tokens_and_spans

In [ ]:
# ## Extract annotations
# ann_path = os.path.join(folder_dir, sentence_name + '.ann')
# entities, relations, attributes, groups = brat_parser.get_entities_relations_attributes_groups(ann_path)

In [ ]:
# entities

In [ ]:
# relations

In [ ]:
def char2tok_span_convert(tokens_and_spans, char_span):
  tok_span = []
  for (i, ts) in enumerate(tokens_and_spans):
    if ts[1] >= char_span[0] and ts[2] <= char_span[1]:
      tok_span.append(i)

  return tok_span

# ## Checking
# char2tok_span_convert(tokens_and_spans, (66, 87))

In [ ]:
def entities_proc(entities, tokens_and_spans):
  ent_dict = {}
  for e in entities.keys():
    ent = entities[e]
    char_span = ent.span[0]
    tok_span = char2tok_span_convert(tokens_and_spans, char_span)
    ent_dict[e] = [ent.type, tok_span]

  return ent_dict

# ## Checking
# entities_proc(entities, tokens_and_spans)  

In [ ]:
def relations_proc(relations):
  rel_dict = {}
  for r in relations.keys():
    rel = relations[r]
    if rel.subj not in rel_dict.keys():
      rel_dict[rel.subj] = [rel.obj]
    else:
      rel_dict[rel.subj].append(rel.obj)

  return rel_dict

# ## Checking
# relations_proc(relations)

In [ ]:
def check_numeral(string):
  try:
      float(string)
      return True
  except ValueError:
      return False

In [ ]:
# entitites_dict = entities_proc(entities, tokens_and_spans)
# relations_dict = relations_proc(relations)

# sentence_examples = []

# for ent_key in entitites_dict.keys():
#   if entitites_dict[ent_key][0] == 'Num' and check_numeral(tokens[entitites_dict[ent_key][1][0]]): 
#     example = {'cont': tokens, 'num': entitites_dict[ent_key][1], 'unit': [], 'targ': []}
#     if ent_key in relations_dict.keys():
#       obj_names = relations_dict[ent_key]
#       for name in obj_names:
#         if entitites_dict[name][0] == 'Unit':
#           example['unit'] = entitites_dict[name][1]
#         elif entitites_dict[name][0] == 'Targ':
#           example['targ'] = entitites_dict[name][1]
#     if example: sentence_examples.append(example)

# sentence_examples

In [ ]:
def extract_examples_from_one_sentence(folder_dir, sentence_name):
  ## Extract content
  txt_path = os.path.join(folder_dir, sentence_name + '.txt')
  f = open(txt_path, 'r')
  content = f.read()

  tokens_and_spans = [t for t in spans(content)]
  # tokens = [t[0] for t in tokens_and_spans]
  tokens = []
  for t in tokens_and_spans:
    token = t[0]
    if token[0].isupper() and n_upper_chars(token) < 2:
      token = token.lower()
    tokens.append(token)  
  tokens = np.array(tokens)

  ## Extract annotations
  ann_path = os.path.join(folder_dir, sentence_name + '.ann')
  entities, relations, _, _ = brat_parser.get_entities_relations_attributes_groups(ann_path) 

  entitites_dict = entities_proc(entities, tokens_and_spans)
  relations_dict = relations_proc(relations)

  ## Generate examples
  sentence_examples = []

  for ent_key in entitites_dict.keys():
    if entitites_dict[ent_key][0] == 'Num' and check_numeral(tokens[entitites_dict[ent_key][1][0]]): 
      example = {'cont': tokens, 'num': entitites_dict[ent_key][1], 'unit': [], 'targ': []}
      if ent_key in relations_dict.keys():
        obj_names = relations_dict[ent_key]
        for name in obj_names:
          if entitites_dict[name][0] == 'Unit':
            example['unit'] = entitites_dict[name][1]
          elif entitites_dict[name][0] == 'Targ':
            example['targ'] = entitites_dict[name][1]
      sentence_examples.append(example)  
  
  return sentence_examples

# ## Checking
# extract_examples_from_one_sentence(folder_dir, sentence_name)

In [ ]:
corpus_examples = []
# N = 0

folder_list = [os.path.join(data_dir, f) for f in sorted(os.listdir(data_dir)) 
               if not os.path.isfile(os.path.join(data_dir, f))]

for (i, folder_dir) in enumerate(folder_list):
  sentence_names = [pathlib.Path(f).stem for f in sorted(os.listdir(folder_dir))
                    if pathlib.Path(f).suffix == '.txt']

  for (j, sentence_name) in enumerate(sentence_names):
    # ann_path = os.path.join(folder_dir, sentence_name + '.ann')
    # entities, relations, _, _ = brat_parser.get_entities_relations_attributes_groups(ann_path)
    # N += len(relations)

    sentence_examples = extract_examples_from_one_sentence(folder_dir, sentence_name)
    if sentence_examples: corpus_examples += sentence_examples # if not empty

In [ ]:
len(corpus_examples), i, folder_dir, j, sentence_name

(1758,
 87,
 '/content/drive/MyDrive/Colab Notebooks/Text2Struct/dataset/annotated/thrombectomy_0-99/PMID_36227897',
 3,
 'S_8')

In [ ]:
for example in corpus_examples:
  if type(example) != dict:
    print(type(example))

In [ ]:
example = corpus_examples[np.random.choice(len(corpus_examples))]
content = example['cont']
print(content)
print('Sentence length is:', len(content))
print()
print('Numeral is:', content[example['num']])
print('Unit is: ', content[example['unit']])
print('Target is:', content[example['targ']])

['there' 'were' '1227' 'patients' '(' '0.499' ')' 'in' 'the' 'DEVT'
 'group' 'and' '1233' 'patients' '(' '0.501' ')' 'in' 'the' 'BT' 'group'
 '.']
Sentence length is: 22

Numeral is: ['1227']
Unit is:  ['patients']
Target is: ['DEVT' 'group']


In [ ]:
## Save data
# filehandler = open(data_dir + "corpus_examples.pkl", "wb")
filehandler = open(out_dir + "corpus_examples.pkl", "wb")
pickle.dump(corpus_examples, filehandler)
filehandler.close()

# Numeral processing

In [ ]:
## Load data
if 'corpus_examples' in globals():
    del corpus_examples

# file = open(data_dir + "corpus_examples.pkl", 'rb')
file = open(out_dir + "corpus_examples.pkl", 'rb')
corpus_examples = pickle.load(file)
file.close()

In [ ]:
def check_numeral(string):
  try:
      float(string)
      return True
  except ValueError:
      return False

In [ ]:
n = np.random.choice(len(corpus_examples))
print(n)
example = corpus_examples[n]

print(example['cont'])
print()
print('Length is:', len(example['cont']))
print('Numeral is:', example['cont'][example['num']])
print('Unit is: ', example['cont'][example['unit']])
print('Target is:', example['cont'][example['targ']])

796
['mean' 'hospital' 'stay' 'were' '(' '3.6' '+/-' '1.7' ')' 'days' '.']

Length is: 11
Numeral is: ['3.6']
Unit is:  ['days']
Target is: ['mean' 'hospital' 'stay']


In [ ]:
# text = example['cont'].copy()
# for (i, w) in enumerate(text):
#   if check_numeral(w) and i != example['num'][0]:
#     text[i] = '[num]'

# text

In [ ]:
# label = np.zeros((len(text),), dtype=int)
# label[example['unit']] = 1
# label[example['targ']] = 2
# label

In [ ]:
# ## Numeral expansion
# # num_string = '-10000'
# num_string = text[example['num']][0]
# num_string = str(float(num_string))
# print(num_string)

# num_chars = []
# for c in num_string:
#   if c == '-':
#     num_chars.append('[neg]')
#   elif c == '.':
#     num_chars.append('[dot]')
#   else:
#     num_chars.append(c)

# num_chars

In [ ]:
# text_before = text[:example['num'][0]]
# print(text_before)

# num_idx = [*range(len(text_before), len(text_before)+len(num_chars), 1)]
# print(num_idx)

# text_after = text[example['num'][0]+1:]
# print(text_after)

# text_new = np.array(list(text_before) + num_chars + list(text_after))
# print(text_new)

In [ ]:
# label_before = label[:example['num'][0]]
# print(label_before)
# label_num = np.zeros((len(num_chars),), dtype=int)
# print(label_num)
# label_after = label[example['num'][0]+1:]
# print(label_after)

# label_new = np.array(list(label_before) + list(label_num) + list(label_after), dtype=int)
# print(label_new)

In [ ]:
# print('Num is:', text_new[num_idx])

# unit_idx = [i for (i, l) in enumerate(label_new) if l == 1]
# print('Unit is:', text_new[unit_idx])

# targ_idx = [i for (i, l) in enumerate(label_new) if l == 2]
# print('Targ is:', text_new[targ_idx])

# print()
# print('num_idx is:', num_idx)
# print('unit_idx is:', unit_idx)
# print('targ_idx is:', targ_idx)

# idxes = num_idx + unit_idx + targ_idx
# print('min-max is:', min(idxes), max(idxes))
# print('range is:', max(idxes) - min(idxes) + 1)

In [ ]:
# numLab = np.zeros((len(text_new),), dtype=int)
# numLab[num_idx] = 1
# print('Num is:', text_new[numLab == 1])

In [ ]:
def proc_numeral_from_one_example(example):
  text = example['cont'].copy()
  for (i, w) in enumerate(text):
    if check_numeral(w) and i != example['num'][0]:
      text[i] = '[num]'

  label = np.zeros((len(text),), dtype=int)
  label[example['unit']] = 1
  label[example['targ']] = 2

  num_string = text[example['num']][0]
  num_string = str(float(num_string))

  num_chars = []
  for c in num_string:
    if c == '-':
      num_chars.append('[neg]')
    elif c == '.':
      num_chars.append('[dot]')
    else:
      num_chars.append(c)

  text_before = text[:example['num'][0]]
  num_idx = [*range(len(text_before), len(text_before)+len(num_chars), 1)]
  text_after = text[example['num'][0]+1:]
  text_new = np.array(list(text_before) + num_chars + list(text_after))

  label_before = label[:example['num'][0]]
  label_num = np.zeros((len(num_chars),), dtype=int)
  label_after = label[example['num'][0]+1:]
  label_new = np.array(list(label_before) + list(label_num) + list(label_after), dtype=int)

  numLab = np.zeros((len(text_new),), dtype=int)
  numLab[num_idx] = 1

  unit_idx = [i for (i, l) in enumerate(label_new) if l == 1]
  targ_idx = [i for (i, l) in enumerate(label_new) if l == 2]

  return {'text': text_new, 'label': label_new, 'numLab': numLab,
          'num_idx': num_idx, 'unit_idx': unit_idx, 'targ_idx': targ_idx}

In [ ]:
example_new = proc_numeral_from_one_example(example)
example_new

{'text': array(['mean', 'hospital', 'stay', 'were', '(', '3', '[dot]', '6', '+/-',
        '[num]', ')', 'days', '.'], dtype='<U8'),
 'label': array([2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]),
 'numLab': array([0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0]),
 'num_idx': [5, 6, 7],
 'unit_idx': [11],
 'targ_idx': [0, 1, 2]}

In [ ]:
corpus_examples_new = []
max_window = 0
for example in corpus_examples:
  example_new = proc_numeral_from_one_example(example)
  corpus_examples_new.append(example_new)

  idxes = example_new['num_idx'] + example_new['unit_idx'] + example_new['targ_idx']
  window = max(idxes) - min(idxes) + 1
  if window > max_window: max_window = window

print(max_window)

41


In [ ]:
corpus_examples_new[n]

{'text': array(['mean', 'hospital', 'stay', 'were', '(', '3', '[dot]', '6', '+/-',
        '[num]', ')', 'days', '.'], dtype='<U8'),
 'label': array([2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]),
 'numLab': array([0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0]),
 'num_idx': [5, 6, 7],
 'unit_idx': [11],
 'targ_idx': [0, 1, 2]}

In [ ]:
## Save data
# filehandler = open(data_dir + "corpus_examples_num_proc.pkl", "wb")
filehandler = open(out_dir + "corpus_examples_num_proc.pkl", "wb")
pickle.dump(corpus_examples_new, filehandler)
filehandler.close()